In [24]:
import math
import uproot
import numpy as np
import matplotlib.pyplot as plt

In [2]:
BASEDIR = '/depot/cms/top/bakshi3/TopSpinCorr_Run2_generalized_ND/CMSSW_10_6_30/src/TopAnalysis/Configuration/analysis/diLeptonic/TUnfoldResults_2016/Nominal/combined/'

In [3]:
var_list = [
'b1k_mttbar',
'b2k_mttbar',
'b1r_mttbar',
'b2r_mttbar',
'b1n_mttbar',
'b2n_mttbar',
'b1j_mttbar',
'b2j_mttbar',
'b1q_mttbar',
'b2q_mttbar',
'c_kk_mttbar',
'c_rr_mttbar',
'c_nn_mttbar',
'c_Prk_mttbar',
'c_Mrk_mttbar',
'c_Pnr_mttbar',
'c_Mnr_mttbar',
'c_Pnk_mttbar',
'c_Mnk_mttbar',
'c_kj_mttbar',
'c_rq_mttbar',
'c_han_mttbar',
'c_tra_mttbar',
'c_sca_mttbar',
'c_Prj_mttbar',
'c_Mrj_mttbar',
'c_kjL_mttbar',
'c_rqL_mttbar',
'c_rkP_mttbar',
'c_rkM_mttbar',
'c_nrP_mttbar',
'c_nrM_mttbar',
'c_nkP_mttbar',
'c_nkM_mttbar',
'll_cHel_mttbar',
'll_cLab_mttbar',
'llbar_delta_phi_mttbar',
'llbar_delta_eta_mttbar',
]

In [4]:
import os

for var in var_list :
    if not (os.path.exists(BASEDIR + "AllSysts_2D_vars_updated_reg/" + str(var) + ".root")) :
        print(var)

In [5]:
fileptr_dict = {}
for var in var_list :
    fileptr_dict[var] = uproot.open(BASEDIR + "AllSysts_2D_vars_updated_reg/" + str(var) + ".root")

## GetDeltasAllVars

In [6]:
def GetDeltasAllVars(SystName, var_list) :
  
    # nbinsrhoi is 6(1D) or 24(2D)  
    # nbinsrhoi = 6
    nbinsrhoi   = 24 
  
    # This constructs a Nominal and Delta vector which is as wide as n_vars * n_bins
    hnom_AllVar = []
    htot_reldelta_AllVar = []
    
    for var in var_list : 
        
        # Unfolded result for particular variable
        hnom = fileptr_dict[var][var + 'TUnfResultCor_rebinnedA'].to_hist().values()
        hnom_AllVar.extend(hnom)
        
        htot_reldelta = [0] * nbinsrhoi
        
        if(SystName=="SCALE" or SystName=="BFRAG" or SystName=="COLORREC")  :

            if (SystName=="SCALE") :    syst_list = ["MESCALE_UP", "MESCALE_DOWN", "MEFACSCALE_UP", "MEFACSCALE_DOWN", "MERENSCALE_UP", "MERENSCALE_DOWN"]
            if (SystName=="BFRAG") :    syst_list = ["BFRAG_UP", "BFRAG_DOWN", "BFRAG_PETERSON" ]
            if (SystName=="COLORREC") : syst_list = ["ERDON", "ERDONRETUNE", "GLUONMOVETUNE" ]
                
            # Envelope
            max_up   = [0] * nbinsrhoi
            max_down = [0] * nbinsrhoi

            for syst in syst_list :
                delta = fileptr_dict[var][var + "RespMatSys_" + syst + "_deltaSys_rebinnedA"].to_hist().values()

                # For each bin, find the maximum up and down variation
                for ibin in range(len(delta)) :
                    max_up[ibin]   = max(max_up[ibin]  , delta[ibin]) 
                    max_down[ibin] = min(max_down[ibin], delta[ibin]) 

                del delta
            
            for ibin in range(len(htot_reldelta)) :
                htot_reldelta[ibin] = (abs(max_up[ibin] - max_down[ibin]) * 0.5)/hnom[ibin]
                

        # Samples with UP and DOWN variations 
        # Jason mentioned not enveloping BTAG and BTAG_LJET
        
        elif( (SystName=="JER")   or ("JES" in SystName)    or (SystName=="PU")          or (SystName=="LEPT")  or 
              (SystName=="KIN")   or (SystName=="BSEMILEP") or (SystName=="UNCLUSTERED") or  (SystName=="TRIG") or
              (SystName=="UETUNE")or (SystName=="MASS")     or (SystName=="MATCH")       or (SystName=="PDF_ALPHAS") or 
              (SystName=="L1PREFIRING")    or (SystName=="ELE_ID")         or (SystName=="ELE_RECO") or 
              (SystName=="ELE_SCALE_GAIN") or (SystName=="ELE_SCALE_SYST") or (SystName=="ELE_SCALE_STAT") or 
              (SystName=="BTAG_CORR")      or (SystName=="BTAG_UNCORR")    or (SystName=="BTAG_LJET_CORR") or (SystName=="BTAG_LJET_UNCORR")or 
              (SystName=="MUON_ID")        or (SystName=="MUON_ISO")       or (SystName=="MUON_SCALE")     or (SystName=="PSSCALE_WEIGHT_6")or 
              (SystName=="PSSCALE_WEIGHT_7")) :

            # This is the upward or downward fluctuation with dimension n_bins
            deltaup    = fileptr_dict[var][var + "RespMatSys_" + SystName + "_UP_deltaSys_rebinnedA"].to_hist().values()
            deltadown  = fileptr_dict[var][var + "RespMatSys_" + SystName + "_DOWN_deltaSys_rebinnedA"].to_hist().values()

            deltafinal = abs(deltaup - deltadown)
            
            for ibin in range(len(htot_reldelta)) :
                htot_reldelta[ibin] = (deltafinal[ibin] * 0.5)/hnom[ibin]
                
            del deltaup
            del deltadown

        # *******************
        # Alternative samples 
        # *******************
        
        elif((SystName=="MCATNLO")  or (SystName=="POWHEG") or (SystName=="POWHEGHERWIG") or 
             (SystName=="POWHEGV2") or (SystName=="POWHEGV2HERWIG") or (SystName=="AMCATNLOFXFX")or 
             (SystName=="TOP_PT")   or (SystName=="MADGRAPHMLM")    or (SystName=="PERUGIA11")   or 
             (SystName=="SPINCORR") or ("PDF" in SystName) ) :

            # This is the fluctuation with dimension n_bins
            deltaSys = fileptr_dict[var][var + "RespMatSys_" + SystName + "_deltaSys_rebinnedA"].to_hist().values()
            
            for ibin in range(len(htot_reldelta)) :
                htot_reldelta[ibin] = (abs(deltaSys[ibin]))/hnom[ibin]
                
            del deltaSys

        # ***********
        # Backgrounds
        # ***********
        
        elif((SystName=="DYeemm") or (SystName=="DYtautau") or (SystName=="singletop") or 
             (SystName=="ww")     or (SystName=="wz")       or (SystName=="zz")     or (SystName=="ttbarW") or 
             (SystName=="ttbarZ") or (SystName=="ttbarbg")  or (SystName=="wtolnu") or (SystName=="other")) :
      
            # This is the fluctuation with dimension n_bins
            bkgSys = fileptr_dict[var][var + "BkgSys_" + SystName + "_deltaSys_rebinnedA"].to_hist().values()
            
            for ibin in range(len(htot_reldelta)) :
                htot_reldelta[ibin] = (abs(bkgSys[ibin]))/hnom[ibin]
                
            del bkgSys
    
        # *****************
        # Flat uncertainties
        # ******************

        elif( (SystName=="Lumi") or (SystName=="BR") ) :
            flat_uncertainty = 0.025 if (SystName=="Lumi") else 0.015

            for ibin in range(len(htot_reldelta)) :
                htot_reldelta[ibin] = flat_uncertainty
            
        else :
            print('Syst :: ' + SystName + ' not supported as of now') 
            return

        htot_reldelta_AllVar.extend(htot_reldelta)

        del hnom;
        del htot_reldelta;
    
    return hnom_AllVar, htot_reldelta_AllVar

In [7]:
# hnom_AllVar, htot_reldelta_AllVar = GetDeltasAllVars("TOP_PT", var_list)
# len(htot_reldelta_AllVar)

## GetCombinedCovarianceMatrixAllVars

In [8]:
def GetCombinedCovarianceMatrixAllVars(hnom_AllVar, htot_reldelta_AllVar, SystName) :

    # Calculate the correlation matrix for the sum of sub-components, 
    # then convert this to a covariance matrix using the envelope results
    
    print("Working with SYST :: " + SystName)

    # nbinsrhoi = 6;  
    nbinsrhoi   = 24;

    htot_absdelta_AllVar = [htot_reldelta_AllVar[i] * hnom_AllVar[i] for i in range(len(hnom_AllVar))]

    syst_list = []
    isBkgSyst = False
    

    if (SystName=="SCALE") :    syst_list = ["MESCALE_UP", "MESCALE_DOWN", "MEFACSCALE_UP", "MEFACSCALE_DOWN", "MERENSCALE_UP", "MERENSCALE_DOWN"]
    if (SystName=="BFRAG") :    syst_list = ["BFRAG_UP", "BFRAG_DOWN", "BFRAG_PETERSON" ]
    if (SystName=="COLORREC") : syst_list = ["ERDON", "ERDONRETUNE", "GLUONMOVETUNE" ]

    # Samples with UP and DOWN variations 
    # Jason mentioned not enveloping BTAG and BTAG_LJET

    elif( (SystName=="JER")   or ("JES" in SystName)    or (SystName=="PU")          or (SystName=="LEPT")  or 
          (SystName=="KIN")   or (SystName=="BSEMILEP") or (SystName=="UNCLUSTERED") or  (SystName=="TRIG") or
          (SystName=="UETUNE")or (SystName=="MASS")     or (SystName=="MATCH")       or (SystName=="PDF_ALPHAS") or 
          (SystName=="L1PREFIRING")    or (SystName=="ELE_ID")         or (SystName=="ELE_RECO") or 
          (SystName=="ELE_SCALE_GAIN") or (SystName=="ELE_SCALE_SYST") or (SystName=="ELE_SCALE_STAT") or 
          (SystName=="BTAG_CORR")      or (SystName=="BTAG_UNCORR")    or (SystName=="BTAG_LJET_CORR") or (SystName=="BTAG_LJET_UNCORR")or 
          (SystName=="MUON_ID")        or (SystName=="MUON_ISO")       or (SystName=="MUON_SCALE")     or (SystName=="PSSCALE_WEIGHT_6")or 
          (SystName=="PSSCALE_WEIGHT_7")) : 
        syst_list = [SystName+"_UP", SystName+"_DOWN"]
  

    # *******************
    # Alternative samples 
    # *******************

    elif((SystName=="MCATNLO")  or (SystName=="POWHEG")         or (SystName=="POWHEGHERWIG") or 
         (SystName=="POWHEGV2") or (SystName=="POWHEGV2HERWIG") or (SystName=="AMCATNLOFXFX")or 
         (SystName=="TOP_PT")   or (SystName=="MADGRAPHMLM")    or (SystName=="PERUGIA11")   or 
         (SystName=="SPINCORR") or ("PDF" in SystName) ) : 
        syst_list = [SystName]
        

    # ***********
    # Backgrounds
    # ***********

    elif((SystName=="DYeemm") or (SystName=="DYtautau") or (SystName=="singletop") or 
         (SystName=="ww")     or (SystName=="wz")       or (SystName=="zz")     or (SystName=="ttbarW") or 
         (SystName=="ttbarZ") or (SystName=="ttbarbg")  or (SystName=="wtolnu") or (SystName=="other")) :
        syst_list = [SystName]
        isBkgSyst = True
        

    scalingfactor = 1.
    if (SystName=="MASS")  : scalingfactor = 1.   # convert from +/-3 GeV to +/-1 GeV
    if ("PDF" in SystName) : scalingfactor = 10.  # convert from sum in quadrature to RMS (for 100 replicas)
    if ("PDF_ALPHAS" in SystName) : scalingfactor = 1.

    systCov = np.zeros((len(htot_absdelta_AllVar), len(htot_absdelta_AllVar)))

    # Loop over systematics in the above described syst_list
    for syst in syst_list : 

        hdelta_AllVar = []
    
        # Over the variables
        for var in var_list :

            # This is the 1D vector with fluctuations of dimension n_bins
            if isBkgSyst :
                delta = fileptr_dict[var][var + "BkgSys_"     + syst + "_deltaSys_rebinnedA"].to_hist().values()
            else :
                delta = fileptr_dict[var][var + "RespMatSys_" + syst + "_deltaSys_rebinnedA"].to_hist().values()

            hdelta_AllVar.extend(delta)
            del delta

        # Add contribution to systCov
        for i in range(len(htot_absdelta_AllVar)) :
            for j in range(len(htot_absdelta_AllVar)) :
                systCov[i][j] += hdelta_AllVar[i]*hdelta_AllVar[j]

        del hdelta_AllVar

    
    # *****************
    # Flat uncertainties
    # ******************

    # Convert to correlation matrix (or fill fully-correlated correlation matrix for the flat systematics)
    if( (SystName=="Lumi") or (SystName=="BR") ) :
        for i in range(len(htot_absdelta_AllVar)) :
            for j in range(len(htot_absdelta_AllVar)) :
                systCov[i][j] = 1

    else :
        # Extract diagonal, replace 0 entries with a small number
        diag_nozeros = [systCov[i][i] if (systCov[i][i] != 0) else 1e-20 for i in range(len(htot_absdelta_AllVar))]

        # This is a replacement for NormByDiag
        # From TMatrixD documentation : b(i,j) = a(i,j)/sqrt(abs*(v(i)*v(j)))
        for i in range(len(htot_absdelta_AllVar)) :
            for j in range(len(htot_absdelta_AllVar)) :
                systCov[i][j] =  systCov[i][j] / math.sqrt(abs(diag_nozeros[i]*diag_nozeros[j]))
                
    
    
    # Convert back to covariance matrix, but using the bin errors from the envelope method
    for i in range(len(htot_absdelta_AllVar)) :
        for j in range(len(htot_absdelta_AllVar)) :
            systCov[i][j] *= abs(htot_absdelta_AllVar[i] * htot_absdelta_AllVar[j])

            
    # New matrix for scaling
    systCovMatrix = np.zeros((len(htot_absdelta_AllVar), len(htot_absdelta_AllVar)))
                                 
    for i in range(len(htot_absdelta_AllVar)) :
        for j in range(len(htot_absdelta_AllVar)) :
            systCovMatrix[i][j] = systCov[i][j] / (scalingfactor**2)

    del htot_absdelta_AllVar
    return systCovMatrix

In [9]:
# hnom_AllVar, htot_reldelta_AllVar = GetDeltasAllVars("PDF_49_UP", var_list)
# GetCombinedCovarianceMatrixAllVars(hnom_AllVar, htot_reldelta_AllVar, "PDF_49_UP")

In [11]:
# Custom Syst List with backgrounds and Lumi, BR added
with open('TUnfoldSystList.txt', 'r') as SystFile :
    AllSysts = SystFile.readlines()

FullSystList = []   

for Syst in AllSysts :
    Syst = Syst.replace('\n','')
    FullSystList.append(Syst) 

In [15]:
TotalSystCovMatrix_AllVar_from_self = np.zeros((912,912))

for Syst in FullSystList :
    hnom_AllVar, htot_reldelta_AllVar = GetDeltasAllVars(Syst, var_list)
    
    SystCovMatrix_AllVar_from_self = GetCombinedCovarianceMatrixAllVars(hnom_AllVar, htot_reldelta_AllVar, Syst)
    SystCovMatrix_AllVar_from_FW   = SystematicsAllVars[Syst + "SystCovMatrix_AllVar_rebinnedA"].to_hist().values()

    # Compare all elements
    if not (SystCovMatrix_AllVar_from_self == SystCovMatrix_AllVar_from_FW).all() :
        print("SYST :: " + Syst + " does not agree with FW calculation")
        continue
    
    TotalSystCovMatrix_AllVar_from_self += SystCovMatrix_AllVar_from_self

Working with SYST :: Lumi
Working with SYST :: BR
Working with SYST :: DYeemm
Working with SYST :: DYtautau
Working with SYST :: singletop
Working with SYST :: ww
Working with SYST :: wz
Working with SYST :: zz
Working with SYST :: ttbarW
Working with SYST :: ttbarZ
Working with SYST :: ttbarbg
Working with SYST :: wtolnu
SYST :: wtolnu does not agree with FW calculation
Working with SYST :: other
Working with SYST :: JER
Working with SYST :: JES
Working with SYST :: UNCLUSTERED
Working with SYST :: PU
Working with SYST :: TRIG
SYST :: TRIG does not agree with FW calculation
Working with SYST :: L1PREFIRING
Working with SYST :: ELE_ID
Working with SYST :: ELE_RECO
Working with SYST :: ELE_SCALE_GAIN
Working with SYST :: ELE_SCALE_STAT
Working with SYST :: ELE_SCALE_SYST
Working with SYST :: MUON_ID
Working with SYST :: MUON_ISO
Working with SYST :: MUON_SCALE
Working with SYST :: BTAG_CORR
Working with SYST :: BTAG_UNCORR
Working with SYST :: BTAG_LJET_CORR
Working with SYST :: BTAG_LJ

SYST :: PDF_41_DOWN does not agree with FW calculation
Working with SYST :: PDF_42_UP
SYST :: PDF_42_UP does not agree with FW calculation
Working with SYST :: PDF_42_DOWN
SYST :: PDF_42_DOWN does not agree with FW calculation
Working with SYST :: PDF_43_UP
SYST :: PDF_43_UP does not agree with FW calculation
Working with SYST :: PDF_43_DOWN
SYST :: PDF_43_DOWN does not agree with FW calculation
Working with SYST :: PDF_44_UP
SYST :: PDF_44_UP does not agree with FW calculation
Working with SYST :: PDF_44_DOWN
SYST :: PDF_44_DOWN does not agree with FW calculation
Working with SYST :: PDF_45_UP
SYST :: PDF_45_UP does not agree with FW calculation
Working with SYST :: PDF_45_DOWN
SYST :: PDF_45_DOWN does not agree with FW calculation
Working with SYST :: PDF_46_UP
SYST :: PDF_46_UP does not agree with FW calculation
Working with SYST :: PDF_46_DOWN
SYST :: PDF_46_DOWN does not agree with FW calculation
Working with SYST :: PDF_47_UP
SYST :: PDF_47_UP does not agree with FW calculation
W

In [31]:
np.savetxt('SystCovMatrix_2D_mttbar_updated_reg.txt', TotalSystCovMatrix_AllVar_from_self)

### Compare to the one obtained from the FW (old)

In [12]:
SystematicsAllVars    = uproot.open(BASEDIR + "AllSysts_2D_vars_updated_reg/Systematics_AllVars.root")
# SystematicsAllVars  = uproot.open(BASEDIR + "Systematics_AllVars_incomplete.root")

In [13]:
[k for k in SystematicsAllVars.keys() if 'TotalSyst' in k]

['TotalSystCovMatrix_AllVar_rebinnedA;1',
 'TotalSystCovMatrix_AllVar_rebinnedANormCalculated;1']

In [16]:
FWCovMatrix = SystematicsAllVars['TotalSystCovMatrix_AllVar_rebinnedA'].to_hist().to_numpy()[0]
FWCovMatrix

array([[ 29.17805946,   5.58309375, -12.18565576, ...,   0.25706303,
          0.15211357,   1.62860577],
       [  5.58309375,  14.1301096 ,  10.22731413, ...,   0.78108302,
          0.74961722,   2.83729556],
       [-12.18565576,  10.22731413,  39.54901183, ...,   0.360693  ,
          1.29799352,   1.42639189],
       ...,
       [  0.25706303,   0.78108302,   0.360693  , ...,   0.4511579 ,
          0.21557413,   0.53833872],
       [  0.15211357,   0.74961722,   1.29799352, ...,   0.21557413,
          0.26829489,   0.44154902],
       [  1.62860577,   2.83729556,   1.42639189, ...,   0.53833872,
          0.44154902,   1.67639401]])

In [17]:
TotalSystCovMatrix_AllVar_from_self

array([[ 29.17777319,   5.58285624, -12.18587967, ...,   0.25706341,
          0.15211471,   1.62862199],
       [  5.58285624,  14.12986227,  10.22707728, ...,   0.78108467,
          0.74962278,   2.83730874],
       [-12.18587967,  10.22707728,  39.54877923, ...,   0.36069603,
          1.29800163,   1.42641166],
       ...,
       [  0.25706341,   0.78108467,   0.36069603, ...,   0.45114133,
          0.2155565 ,   0.53828162],
       [  0.15211471,   0.74962278,   1.29800163, ...,   0.2155565 ,
          0.26827413,   0.44148088],
       [  1.62862199,   2.83730874,   1.42641166, ...,   0.53828162,
          0.44148088,   1.67614179]])

In [25]:
selfdiag = np.array([TotalSystCovMatrix_AllVar_from_self[i][i] for i in range(912)])
FWdiag   = np.array([FWCovMatrix[i][i] for i in range(912)])

(selfdiag == FWdiag).all()

False

In [27]:
np.allclose(selfdiag, FWdiag, atol=1e-2)

True

## Now let's normalize

Consider an observable $x$ and its covariance matrix $\Sigma_x$. The generalized covariance transformation is $\Sigma_y = J \Sigma_x J^T$ where $J$ is the Jacobian matrix:

$J = 
\begin{bmatrix} 
\partial f_0 / \partial x_0 & \partial f_0 / \partial x_1 & \ldots \\
\partial f_1 / \partial x_0 & \partial f_1 / \partial x_1 & \ldots \\
\ldots & & \\
\end{bmatrix}$

Now consider a transformation of $x$ to normalize it: 

$y = \frac{x}{ \sum_i x_i} \equiv \frac{x}{N}$,


Then the Jacobian is,

$J = 
\begin{bmatrix} 
\frac{N-x_0}{N^2}  & \frac{-x_0}{N^2} & \frac{-x_0}{N^2} & \ldots \\
\frac{-x_1}{N^2} & \frac{N-x_1}{N^2} & \frac{-x_1}{N^2} & \ldots \\
\ldots 
\end{bmatrix}$


Coordinate transformations change the uncertainties as,

$\sigma_y^2 = J \sigma_x^2 J^T$

where $\sigma_x$ is the covariance matrix, and $y$ is defined above. 

In [32]:
nbinsrhoi   = 24
nbinstot    = nbinsrhoi * len(var_list)
sum_AllVar  = []
hnom_AllVar = []

for var in var_list : 

    # Unfolded result for particular variable
    hnom = fileptr_dict[var][var + 'TUnfResultCor_rebinnedA'].to_hist().values()
    sum_AllVar.append(sum(hnom))
    hnom_AllVar.append(hnom)     # careful to append and not extend
    
dfdn  = np.zeros((nbinstot, nbinstot))

for i in range(nbinstot) :
    for j in range(nbinstot) :
        if (i == j) : 
            sum_n =  sum_AllVar[i//nbinsrhoi] # i//nbinsrhoi is var number, so get integral for that variable
            unf_i = hnom_AllVar[i//nbinsrhoi][i%nbinsrhoi] # i%nbinsrhoi is bin number, so list <var> <bin>
            
            dfdn[i][j]  = (sum_n - unf_i) / sum_n**2
            del sum_n, unf_i
            
        elif (i//nbinsrhoi == j//nbinsrhoi) :
            sum_n =  sum_AllVar[i//nbinsrhoi] # i//nbinsrhoi is var number, so get integral for that variable
            unf_i = hnom_AllVar[i//nbinsrhoi][i%nbinsrhoi] # i%nbinsrhoi is bin number, so list <var> <bin>
            
            dfdn[i][j]  = (- unf_i) / sum_n**2
            del sum_n, unf_i
            
        else :
            dfdn[i][j] = 0

In [33]:
TotalSystCovMatrixNorm_AllVar_from_self = np.matmul(np.matmul(dfdn, TotalSystCovMatrix_AllVar_from_self), dfdn.T)
TotalSystCovMatrixNorm_AllVar_from_self

array([[ 3.74784741e-05,  9.84304253e-07, -2.98716233e-05, ...,
        -3.38928557e-07, -9.95002643e-07, -4.89689284e-07],
       [ 9.84304253e-07,  1.18721843e-05,  1.75868406e-06, ...,
         3.33697817e-07, -1.81233055e-07,  1.07457987e-06],
       [-2.98716233e-05,  1.75868406e-06,  4.05612535e-05, ...,
        -7.60887230e-07,  1.38522985e-07, -2.37371907e-06],
       ...,
       [-3.38928557e-07,  3.33697817e-07, -7.60887230e-07, ...,
         6.39681606e-07,  2.39717998e-07,  6.39245574e-07],
       [-9.95002643e-07, -1.81233055e-07,  1.38522985e-07, ...,
         2.39717998e-07,  2.61361282e-07,  3.44223404e-07],
       [-4.89689284e-07,  1.07457987e-06, -2.37371907e-06, ...,
         6.39245574e-07,  3.44223404e-07,  1.81836688e-06]])

In [34]:
np.savetxt('SystCovMatrixNorm_2D_mttbar_updated_reg.txt', TotalSystCovMatrixNorm_AllVar_from_self)